# Unit 5 Lecture 2 - Wrangling Data

CAI 4823: Artificial Intelligence for Industrial Engineers<br>
University of Central Florida


## Notebook Learning Objectives
After studying this notebook students should be able to:
- Reshaping data using stack and unstack index data hierarchically
- Group, aggregate, and transform data
- merge datasets using inner, outer, left and right join operations
- create Pivot tables
- remove NaN from data and remove duplicates

# Overview

When the large amounts of data are spread across various files, accessing that data is lot easier when organized properly. In this chapter we will learn how to aggregate the data by wrangling as needed. 

In [1]:
import pandas as pd
import numpy as np

# Hierarchical Indexing

Dealing with higher dimensional data is always a challenge, so we need to index them hierarchically, this helps in working with higher dimensional data in lower dimensional form, also known as hierarchical indexing. Lets begin with an simple example. 

To create a hierarchical index, we simply define the index of the dataframe as two or more lists as follows:

In [2]:
data = pd.DataFrame(np.random.randn(27).reshape(9,3), #create random data for frame
                    index = [
                                ['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd' ],
                                [  1,   2,   3,   1,   3,   1,   2,   2,   3]
                            ]
                   )
data

0         1         2
a 1  0.557197 -0.468306  1.093532
  2  1.393334  0.302445  0.938368
  3 -0.343116 -1.541474  1.205195
b 1  1.552512  0.153540 -0.190169
  3  1.423552  0.467994 -1.478787
c 1 -0.218470  1.563943  0.365676
  2 -0.827151 -0.327079  0.637376
d 2 -0.379316  0.427941 -0.519877
  3  0.067118  0.363220 -0.677648

A hierarchical index can be also at the column indexes:

In [3]:
data = pd.DataFrame(np.random.randn(27).reshape(9,3), 
                    index = [
                                ['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd' ], 
                                [1, 2, 3, 1, 3, 1, 2, 2, 3]
                            ], 
                    columns = [
                                ['Ohio', 'Ohio', 'Colorado'], 
                                ['Green', 'Red', 'Green']
                              ]
                   )
data

Ohio            Colorado
        Green       Red     Green
a 1  0.534009  1.039794  0.627021
  2 -0.947370  1.194888 -0.872836
  3  1.038612 -0.426701 -0.958991
b 1 -3.593551  0.051974 -0.656949
  3 -2.611949  0.567590 -0.354348
c 1 -0.650940  0.645641  1.254981
  2 -0.674695  0.895226  0.413842
d 2  0.160576 -1.712097 -0.588128
  3 -1.176400 -1.022140 -0.215307

# Reshaping Data Frames

We use <code>.stack()</code> and <code>.unstack()</code> methods to reshape the Data Frame by exchanging row for column indexes. Lets load the following Excel file containing financial data per quarter of three big tech firms to illustrate.
**Note:** I am using <code>.read_excel()</code> to read the Excel file. I am providing _Excel file location_, _sheet name inside the file_, and that _first and second rows are headers_ not data. **Header** = Row (0-indexed) to use for the column labels of the parsed DataFrame. If a list of integers is passed those row positions will be combined into a MultiIndex. 

## <code>.stack()</code> and <code>.unstack()</code>

In [4]:
findata = pd.read_excel('../Data/DSS-Unit05-File01.xlsx', 
                        sheet_name = 'Financials_v1', 
                        header = [0,1]) #header: If a list of integers is passed those row 
                                        #positions will be combined into a MultiIndex. 
findata

Unnamed: 0_level_0 Sales/Revenue                 Gross Income/Profit         \
  Unnamed: 0_level_1         Apple Amazon Alphabet               Apple Amazon   
0         2018-06-30         53427  52886    32758               20789  22254   
1         2018-03-31         61224  51042    31393               23530  20307   
2         2017-12-31         88477  60453    32521               34069  21959   
3         2017-09-30         52574  43744    27963               19902  16195   
4         2017-06-30         45260  37955    26007               17267  14504   

           Total Assets                  Total Liabilities                   
  Alphabet        Apple  Amazon Alphabet             Apple  Amazon Alphabet  
0    18875       375319  134100   197295            241272   99105    44793  
1    17926       321686  126362   167497            193437   94899    28461  
2    18254       290479  131310   147461            171124  103601    27130  
3    16815       231839  115267   129187            120292   90609    25327  
4    15634       207000   87781   110920             83451   64567    23611

# Now lets use <code>.stack()</code> and see what happens:

In [5]:
stacked_findata = findata.stack()
stacked_findata

Gross Income/Profit  Sales/Revenue  Total Assets  \
0 Alphabet                        18875.0        32758.0      197295.0   
  Amazon                          22254.0        52886.0      134100.0   
  Apple                           20789.0        53427.0      375319.0   
  Unnamed: 0_level_1                  NaN            NaN           NaN   
1 Alphabet                        17926.0        31393.0      167497.0   
  Amazon                          20307.0        51042.0      126362.0   
  Apple                           23530.0        61224.0      321686.0   
  Unnamed: 0_level_1                  NaN            NaN           NaN   
2 Alphabet                        18254.0        32521.0      147461.0   
  Amazon                          21959.0        60453.0      131310.0   
  Apple                           34069.0        88477.0      290479.0   
  Unnamed: 0_level_1                  NaN            NaN           NaN   
3 Alphabet                        16815.0        27963.0      129187.0   
  Amazon                          16195.0        43744.0      115267.0   
  Apple                           19902.0        52574.0      231839.0   
  Unnamed: 0_level_1                  NaN            NaN           NaN   
4 Alphabet                        15634.0        26007.0      110920.0   
  Amazon                          14504.0        37955.0       87781.0   
  Apple                           17267.0        45260.0      207000.0   
  Unnamed: 0_level_1                  NaN            NaN           NaN   

                      Total Liabilities Unnamed: 0_level_0  
0 Alphabet                      44793.0                NaT  
  Amazon                        99105.0                NaT  
  Apple                        241272.0                NaT  
  Unnamed: 0_level_1                NaN         2018-06-30  
1 Alphabet                      28461.0                NaT  
  Amazon                        94899.0                NaT  
  Apple                        193437.0                NaT  
  Unnamed: 0_level_1                NaN         2018-03-31  
2 Alphabet                      27130.0                NaT  
  Amazon                       103601.0                NaT  
  Apple                        171124.0                NaT  
  Unnamed: 0_level_1                NaN         2017-12-31  
3 Alphabet                      25327.0                NaT  
  Amazon                        90609.0                NaT  
  Apple                        120292.0                NaT  
  Unnamed: 0_level_1                NaN         2017-09-30  
4 Alphabet                      23611.0                NaT  
  Amazon                        64567.0                NaT  
  Apple                         83451.0                NaT  
  Unnamed: 0_level_1                NaN         2017-06-30

The inner most column index (Apple, Amazon, Alphabet) was transposed from columns into rows. Now lets try to reverse this operation with <code>.unstack()</code>

In [6]:
stacked_findata.unstack() # operation is reversible

Gross Income/Profit                                      Sales/Revenue  \
             Alphabet   Amazon    Apple Unnamed: 0_level_1      Alphabet   
0             18875.0  22254.0  20789.0                NaN       32758.0   
1             17926.0  20307.0  23530.0                NaN       31393.0   
2             18254.0  21959.0  34069.0                NaN       32521.0   
3             16815.0  16195.0  19902.0                NaN       27963.0   
4             15634.0  14504.0  17267.0                NaN       26007.0   

                                       Total Assets                      \
    Amazon    Apple Unnamed: 0_level_1     Alphabet    Amazon     Apple   
0  52886.0  53427.0                NaN     197295.0  134100.0  375319.0   
1  51042.0  61224.0                NaN     167497.0  126362.0  321686.0   
2  60453.0  88477.0                NaN     147461.0  131310.0  290479.0   
3  43744.0  52574.0                NaN     129187.0  115267.0  231839.0   
4  37955.0  45260.0                NaN     110920.0   87781.0  207000.0   

                     Total Liabilities                                         \
  Unnamed: 0_level_1          Alphabet    Amazon     Apple Unnamed: 0_level_1   
0                NaN           44793.0   99105.0  241272.0                NaN   
1                NaN           28461.0   94899.0  193437.0                NaN   
2                NaN           27130.0  103601.0  171124.0                NaN   
3                NaN           25327.0   90609.0  120292.0                NaN   
4                NaN           23611.0   64567.0   83451.0                NaN   

  Unnamed: 0_level_0                                  
            Alphabet Amazon Apple Unnamed: 0_level_1  
0                NaT    NaT   NaT         2018-06-30  
1                NaT    NaT   NaT         2018-03-31  
2                NaT    NaT   NaT         2017-12-31  
3                NaT    NaT   NaT         2017-09-30  
4                NaT    NaT   NaT         2017-06-30

Lets load another example from the same Excel file. This example adds a third index for columns:

In [7]:
findata2 = pd.read_excel('../Data/DSS-Unit05-File01.xlsx', 
                         sheet_name = 'Financials_v3',
                         header = [0,1,2]) #importing data from an Excel file
findata2

Unnamed: 0_level_0 Income Statement                                      \
  Unnamed: 0_level_1    Sales/Revenue                 Gross Income/Profit   
  Unnamed: 0_level_2            Apple Amazon Alphabet               Apple   
0         2018-06-30            53427  52886    32758               20789   
1         2018-03-31            61224  51042    31393               23530   
2         2017-12-31            88477  60453    32521               34069   
3         2017-09-30            52574  43744    27963               19902   
4         2017-06-30            45260  37955    26007               17267   

                  Balance Sheet                                             \
                   Total Assets                  Total Liabilities           
  Amazon Alphabet         Apple  Amazon Alphabet             Apple  Amazon   
0  22254    18875        375319  134100   197295            241272   99105   
1  20307    17926        321686  126362   167497            193437   94899   
2  21959    18254        290479  131310   147461            171124  103601   
3  16195    16815        231839  115267   129187            120292   90609   
4  14504    15634        207000   87781   110920             83451   64567   

            
            
  Alphabet  
0    44793  
1    28461  
2    27130  
3    25327  
4    23611

In [8]:
findata2.stack()

Balance Sheet                      Income Statement  \
                      Total Assets Total Liabilities Gross Income/Profit   
0 Alphabet                197295.0           44793.0             18875.0   
  Amazon                  134100.0           99105.0             22254.0   
  Apple                   375319.0          241272.0             20789.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   
1 Alphabet                167497.0           28461.0             17926.0   
  Amazon                  126362.0           94899.0             20307.0   
  Apple                   321686.0          193437.0             23530.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   
2 Alphabet                147461.0           27130.0             18254.0   
  Amazon                  131310.0          103601.0             21959.0   
  Apple                   290479.0          171124.0             34069.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   
3 Alphabet                129187.0           25327.0             16815.0   
  Amazon                  115267.0           90609.0             16195.0   
  Apple                   231839.0          120292.0             19902.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   
4 Alphabet                110920.0           23611.0             15634.0   
  Amazon                   87781.0           64567.0             14504.0   
  Apple                   207000.0           83451.0             17267.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   

                                   Unnamed: 0_level_0  
                     Sales/Revenue Unnamed: 0_level_1  
0 Alphabet                 32758.0                NaT  
  Amazon                   52886.0                NaT  
  Apple                    53427.0                NaT  
  Unnamed: 0_level_2           NaN         2018-06-30  
1 Alphabet                 31393.0                NaT  
  Amazon                   51042.0                NaT  
  Apple                    61224.0                NaT  
  Unnamed: 0_level_2           NaN         2018-03-31  
2 Alphabet                 32521.0                NaT  
  Amazon                   60453.0                NaT  
  Apple                    88477.0                NaT  
  Unnamed: 0_level_2           NaN         2017-12-31  
3 Alphabet                 27963.0                NaT  
  Amazon                   43744.0                NaT  
  Apple                    52574.0                NaT  
  Unnamed: 0_level_2           NaN         2017-09-30  
4 Alphabet                 26007.0                NaT  
  Amazon                   37955.0                NaT  
  Apple                    45260.0                NaT  
  Unnamed: 0_level_2           NaN         2017-06-30

In [9]:
findata2.stack(level=2) 
#level 2 is the "inner-most" default, so we obtain the same result as above

Balance Sheet                      Income Statement  \
                      Total Assets Total Liabilities Gross Income/Profit   
0 Alphabet                197295.0           44793.0             18875.0   
  Amazon                  134100.0           99105.0             22254.0   
  Apple                   375319.0          241272.0             20789.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   
1 Alphabet                167497.0           28461.0             17926.0   
  Amazon                  126362.0           94899.0             20307.0   
  Apple                   321686.0          193437.0             23530.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   
2 Alphabet                147461.0           27130.0             18254.0   
  Amazon                  131310.0          103601.0             21959.0   
  Apple                   290479.0          171124.0             34069.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   
3 Alphabet                129187.0           25327.0             16815.0   
  Amazon                  115267.0           90609.0             16195.0   
  Apple                   231839.0          120292.0             19902.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   
4 Alphabet                110920.0           23611.0             15634.0   
  Amazon                   87781.0           64567.0             14504.0   
  Apple                   207000.0           83451.0             17267.0   
  Unnamed: 0_level_2           NaN               NaN                 NaN   

                                   Unnamed: 0_level_0  
                     Sales/Revenue Unnamed: 0_level_1  
0 Alphabet                 32758.0                NaT  
  Amazon                   52886.0                NaT  
  Apple                    53427.0                NaT  
  Unnamed: 0_level_2           NaN         2018-06-30  
1 Alphabet                 31393.0                NaT  
  Amazon                   51042.0                NaT  
  Apple                    61224.0                NaT  
  Unnamed: 0_level_2           NaN         2018-03-31  
2 Alphabet                 32521.0                NaT  
  Amazon                   60453.0                NaT  
  Apple                    88477.0                NaT  
  Unnamed: 0_level_2           NaN         2017-12-31  
3 Alphabet                 27963.0                NaT  
  Amazon                   43744.0                NaT  
  Apple                    52574.0                NaT  
  Unnamed: 0_level_2           NaN         2017-09-30  
4 Alphabet                 26007.0                NaT  
  Amazon                   37955.0                NaT  
  Apple                    45260.0                NaT  
  Unnamed: 0_level_2           NaN         2017-06-30

In [10]:
findata2.stack(level=0) 
#level 0 is the "financial report" level, this time that level gets "stacked"

Gross Income/Profit                   Sales/Revenue  \
                                Alphabet   Amazon    Apple      Alphabet   
0 Balance Sheet                      NaN      NaN      NaN           NaN   
  Income Statement               18875.0  22254.0  20789.0       32758.0   
  Unnamed: 0_level_0                 NaN      NaN      NaN           NaN   
1 Balance Sheet                      NaN      NaN      NaN           NaN   
  Income Statement               17926.0  20307.0  23530.0       31393.0   
  Unnamed: 0_level_0                 NaN      NaN      NaN           NaN   
2 Balance Sheet                      NaN      NaN      NaN           NaN   
  Income Statement               18254.0  21959.0  34069.0       32521.0   
  Unnamed: 0_level_0                 NaN      NaN      NaN           NaN   
3 Balance Sheet                      NaN      NaN      NaN           NaN   
  Income Statement               16815.0  16195.0  19902.0       27963.0   
  Unnamed: 0_level_0                 NaN      NaN      NaN           NaN   
4 Balance Sheet                      NaN      NaN      NaN           NaN   
  Income Statement               15634.0  14504.0  17267.0       26007.0   
  Unnamed: 0_level_0                 NaN      NaN      NaN           NaN   

                                       Total Assets                      \
                       Amazon    Apple     Alphabet    Amazon     Apple   
0 Balance Sheet           NaN      NaN     197295.0  134100.0  375319.0   
  Income Statement    52886.0  53427.0          NaN       NaN       NaN   
  Unnamed: 0_level_0      NaN      NaN          NaN       NaN       NaN   
1 Balance Sheet           NaN      NaN     167497.0  126362.0  321686.0   
  Income Statement    51042.0  61224.0          NaN       NaN       NaN   
  Unnamed: 0_level_0      NaN      NaN          NaN       NaN       NaN   
2 Balance Sheet           NaN      NaN     147461.0  131310.0  290479.0   
  Income Statement    60453.0  88477.0          NaN       NaN       NaN   
  Unnamed: 0_level_0      NaN      NaN          NaN       NaN       NaN   
3 Balance Sheet           NaN      NaN     129187.0  115267.0  231839.0   
  Income Statement    43744.0  52574.0          NaN       NaN       NaN   
  Unnamed: 0_level_0      NaN      NaN          NaN       NaN       NaN   
4 Balance Sheet           NaN      NaN     110920.0   87781.0  207000.0   
  Income Statement    37955.0  45260.0          NaN       NaN       NaN   
  Unnamed: 0_level_0      NaN      NaN          NaN       NaN       NaN   

                     Total Liabilities                     Unnamed: 0_level_1  
                              Alphabet    Amazon     Apple Unnamed: 0_level_2  
0 Balance Sheet                44793.0   99105.0  241272.0                NaT  
  Income Statement                 NaN       NaN       NaN                NaT  
  Unnamed: 0_level_0               NaN       NaN       NaN         2018-06-30  
1 Balance Sheet                28461.0   94899.0  193437.0                NaT  
  Income Statement                 NaN       NaN       NaN                NaT  
  Unnamed: 0_level_0               NaN       NaN       NaN         2018-03-31  
2 Balance Sheet                27130.0  103601.0  171124.0                NaT  
  Income Statement                 NaN       NaN       NaN                NaT  
  Unnamed: 0_level_0               NaN       NaN       NaN         2017-12-31  
3 Balance Sheet                25327.0   90609.0  120292.0                NaT  
  Income Statement                 NaN       NaN       NaN                NaT  
  Unnamed: 0_level_0               NaN       NaN       NaN         2017-09-30  
4 Balance Sheet                23611.0   64567.0   83451.0                NaT  
  Income Statement                 NaN       NaN       NaN                NaT  
  Unnamed: 0_level_0               NaN       NaN       NaN         2017-06-30

In [11]:
findata2.stack().stack() # lets stack twice, two inner-most levels get stacked.

Balance Sheet  Income Statement  \
0 Alphabet           Gross Income/Profit            NaN           18875.0   
                     Sales/Revenue                  NaN           32758.0   
                     Total Assets              197295.0               NaN   
                     Total Liabilities          44793.0               NaN   
  Amazon             Gross Income/Profit            NaN           22254.0   
...                                                 ...               ...   
4 Apple              Gross Income/Profit            NaN           17267.0   
                     Sales/Revenue                  NaN           45260.0   
                     Total Assets              207000.0               NaN   
                     Total Liabilities          83451.0               NaN   
  Unnamed: 0_level_2 Unnamed: 0_level_1             NaN               NaN   

                                         Unnamed: 0_level_0  
0 Alphabet           Gross Income/Profit                NaT  
                     Sales/Revenue                      NaT  
                     Total Assets                       NaT  
                     Total Liabilities                  NaT  
  Amazon             Gross Income/Profit                NaT  
...                                                     ...  
4 Apple              Gross Income/Profit                NaT  
                     Sales/Revenue                      NaT  
                     Total Assets                       NaT  
                     Total Liabilities                  NaT  
  Unnamed: 0_level_2 Unnamed: 0_level_1          2017-06-30  

[65 rows x 3 columns]

## <code>.swaplevel()</code> and <code>.sort_index()</code>

These methods are used to rearrange the order of the levels on an axis or sort the data by the values in one specific level. 

* .swaplevel() : The swaplevel takes two level numbers or names and returns a new object with the levels interchanged. 
* .sort_index() : This method sorts the data using only the values in a single level. <br>
** Note: ** When swapping levels, it is not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level. For instance, <code>.swaplevel(0,1).sort_index(level=0)</code>. 


In [12]:
findata3 = findata2.stack(level=1)
findata3

Balance Sheet                     Income Statement  \
                           Alphabet    Amazon     Apple         Alphabet   
0 Gross Income/Profit           NaN       NaN       NaN          18875.0   
  Sales/Revenue                 NaN       NaN       NaN          32758.0   
  Total Assets             197295.0  134100.0  375319.0              NaN   
  Total Liabilities         44793.0   99105.0  241272.0              NaN   
  Unnamed: 0_level_1            NaN       NaN       NaN              NaN   
1 Gross Income/Profit           NaN       NaN       NaN          17926.0   
  Sales/Revenue                 NaN       NaN       NaN          31393.0   
  Total Assets             167497.0  126362.0  321686.0              NaN   
  Total Liabilities         28461.0   94899.0  193437.0              NaN   
  Unnamed: 0_level_1            NaN       NaN       NaN              NaN   
2 Gross Income/Profit           NaN       NaN       NaN          18254.0   
  Sales/Revenue                 NaN       NaN       NaN          32521.0   
  Total Assets             147461.0  131310.0  290479.0              NaN   
  Total Liabilities         27130.0  103601.0  171124.0              NaN   
  Unnamed: 0_level_1            NaN       NaN       NaN              NaN   
3 Gross Income/Profit           NaN       NaN       NaN          16815.0   
  Sales/Revenue                 NaN       NaN       NaN          27963.0   
  Total Assets             129187.0  115267.0  231839.0              NaN   
  Total Liabilities         25327.0   90609.0  120292.0              NaN   
  Unnamed: 0_level_1            NaN       NaN       NaN              NaN   
4 Gross Income/Profit           NaN       NaN       NaN          15634.0   
  Sales/Revenue                 NaN       NaN       NaN          26007.0   
  Total Assets             110920.0   87781.0  207000.0              NaN   
  Total Liabilities         23611.0   64567.0   83451.0              NaN   
  Unnamed: 0_level_1            NaN       NaN       NaN              NaN   

                                        Unnamed: 0_level_0  
                        Amazon    Apple Unnamed: 0_level_2  
0 Gross Income/Profit  22254.0  20789.0                NaT  
  Sales/Revenue        52886.0  53427.0                NaT  
  Total Assets             NaN      NaN                NaT  
  Total Liabilities        NaN      NaN                NaT  
  Unnamed: 0_level_1       NaN      NaN         2018-06-30  
1 Gross Income/Profit  20307.0  23530.0                NaT  
  Sales/Revenue        51042.0  61224.0                NaT  
  Total Assets             NaN      NaN                NaT  
  Total Liabilities        NaN      NaN                NaT  
  Unnamed: 0_level_1       NaN      NaN         2018-03-31  
2 Gross Income/Profit  21959.0  34069.0                NaT  
  Sales/Revenue        60453.0  88477.0                NaT  
  Total Assets             NaN      NaN                NaT  
  Total Liabilities        NaN      NaN                NaT  
  Unnamed: 0_level_1       NaN      NaN         2017-12-31  
3 Gross Income/Profit  16195.0  19902.0                NaT  
  Sales/Revenue        43744.0  52574.0                NaT  
  Total Assets             NaN      NaN                NaT  
  Total Liabilities        NaN      NaN                NaT  
  Unnamed: 0_level_1       NaN      NaN         2017-09-30  
4 Gross Income/Profit  14504.0  17267.0                NaT  
  Sales/Revenue        37955.0  45260.0                NaT  
  Total Assets             NaN      NaN                NaT  
  Total Liabilities        NaN      NaN                NaT  
  Unnamed: 0_level_1       NaN      NaN         2017-06-30

In [13]:
findata3.stack()

Balance Sheet  Income Statement  \
0 Gross Income/Profit Alphabet                      NaN           18875.0   
                      Amazon                        NaN           22254.0   
                      Apple                         NaN           20789.0   
  Sales/Revenue       Alphabet                      NaN           32758.0   
                      Amazon                        NaN           52886.0   
...                                                 ...               ...   
4 Total Assets        Apple                    207000.0               NaN   
  Total Liabilities   Alphabet                  23611.0               NaN   
                      Amazon                    64567.0               NaN   
                      Apple                     83451.0               NaN   
  Unnamed: 0_level_1  Unnamed: 0_level_2            NaN               NaN   

                                         Unnamed: 0_level_0  
0 Gross Income/Profit Alphabet                          NaT  
                      Amazon                            NaT  
                      Apple                             NaT  
  Sales/Revenue       Alphabet                          NaT  
                      Amazon                            NaT  
...                                                     ...  
4 Total Assets        Apple                             NaT  
  Total Liabilities   Alphabet                          NaT  
                      Amazon                            NaT  
                      Apple                             NaT  
  Unnamed: 0_level_1  Unnamed: 0_level_2         2017-06-30  

[65 rows x 3 columns]

In [14]:
findata3.stack().swaplevel(0,1)

Balance Sheet  Income Statement  \
Gross Income/Profit 0 Alphabet                      NaN           18875.0   
                      Amazon                        NaN           22254.0   
                      Apple                         NaN           20789.0   
Sales/Revenue       0 Alphabet                      NaN           32758.0   
                      Amazon                        NaN           52886.0   
...                                                 ...               ...   
Total Assets        4 Apple                    207000.0               NaN   
Total Liabilities   4 Alphabet                  23611.0               NaN   
                      Amazon                    64567.0               NaN   
                      Apple                     83451.0               NaN   
Unnamed: 0_level_1  4 Unnamed: 0_level_2            NaN               NaN   

                                         Unnamed: 0_level_0  
Gross Income/Profit 0 Alphabet                          NaT  
                      Amazon                            NaT  
                      Apple                             NaT  
Sales/Revenue       0 Alphabet                          NaT  
                      Amazon                            NaT  
...                                                     ...  
Total Assets        4 Apple                             NaT  
Total Liabilities   4 Alphabet                          NaT  
                      Amazon                            NaT  
                      Apple                             NaT  
Unnamed: 0_level_1  4 Unnamed: 0_level_2         2017-06-30  

[65 rows x 3 columns]

In [15]:
findata3.stack().swaplevel(0,1).head(10)

Balance Sheet  Income Statement  \
Gross Income/Profit 0 Alphabet            NaN           18875.0   
                      Amazon              NaN           22254.0   
                      Apple               NaN           20789.0   
Sales/Revenue       0 Alphabet            NaN           32758.0   
                      Amazon              NaN           52886.0   
                      Apple               NaN           53427.0   
Total Assets        0 Alphabet       197295.0               NaN   
                      Amazon         134100.0               NaN   
                      Apple          375319.0               NaN   
Total Liabilities   0 Alphabet        44793.0               NaN   

                               Unnamed: 0_level_0  
Gross Income/Profit 0 Alphabet                NaT  
                      Amazon                  NaT  
                      Apple                   NaT  
Sales/Revenue       0 Alphabet                NaT  
                      Amazon                  NaT  
                      Apple                   NaT  
Total Assets        0 Alphabet                NaT  
                      Amazon                  NaT  
                      Apple                   NaT  
Total Liabilities   0 Alphabet                NaT

In [58]:
findata3.stack().swaplevel(0,1).sort_index(level=0).head(60)

Balance Sheet  Income Statement  \
Gross Income/Profit 0 Alphabet            NaN           18875.0   
                      Amazon              NaN           22254.0   
                      Apple               NaN           20789.0   
                    1 Alphabet            NaN           17926.0   
                      Amazon              NaN           20307.0   
                      Apple               NaN           23530.0   
                    2 Alphabet            NaN           18254.0   
                      Amazon              NaN           21959.0   
                      Apple               NaN           34069.0   
                    3 Alphabet            NaN           16815.0   
                      Amazon              NaN           16195.0   
                      Apple               NaN           19902.0   
                    4 Alphabet            NaN           15634.0   
                      Amazon              NaN           14504.0   
                      Apple               NaN           17267.0   
Sales/Revenue       0 Alphabet            NaN           32758.0   
                      Amazon              NaN           52886.0   
                      Apple               NaN           53427.0   
                    1 Alphabet            NaN           31393.0   
                      Amazon              NaN           51042.0   
                      Apple               NaN           61224.0   
                    2 Alphabet            NaN           32521.0   
                      Amazon              NaN           60453.0   
                      Apple               NaN           88477.0   
                    3 Alphabet            NaN           27963.0   
                      Amazon              NaN           43744.0   
                      Apple               NaN           52574.0   
                    4 Alphabet            NaN           26007.0   
                      Amazon              NaN           37955.0   
                      Apple               NaN           45260.0   
Total Assets        0 Alphabet       197295.0               NaN   
                      Amazon         134100.0               NaN   
                      Apple          375319.0               NaN   
                    1 Alphabet       167497.0               NaN   
                      Amazon         126362.0               NaN   
                      Apple          321686.0               NaN   
                    2 Alphabet       147461.0               NaN   
                      Amazon         131310.0               NaN   
                      Apple          290479.0               NaN   
                    3 Alphabet       129187.0               NaN   
                      Amazon         115267.0               NaN   
                      Apple          231839.0               NaN   
                    4 Alphabet       110920.0               NaN   
                      Amazon          87781.0               NaN   
                      Apple          207000.0               NaN   
Total Liabilities   0 Alphabet        44793.0               NaN   
                      Amazon          99105.0               NaN   
                      Apple          241272.0               NaN   
                    1 Alphabet        28461.0               NaN   
                      Amazon          94899.0               NaN   
                      Apple          193437.0               NaN   
                    2 Alphabet        27130.0               NaN   
                      Amazon         103601.0               NaN   
                      Apple          171124.0               NaN   
                    3 Alphabet        25327.0               NaN   
                      Amazon          90609.0               NaN   
                      Apple          120292.0               NaN   
                    4 Alphabet        23611.0               NaN   
                      Amazon          64567.0               NaN   
            

# Aggregate, Group, Filter and Transform Data

Aggregation Methods:
* df.count() : This Method counts total number of items
* df.first(), df.last() : Methods to get first and last item respectively.
* df.mean(), df.median() : Methods to get mean and median respectively.
* df. min(), df.max() : Methods to get minimum and maximum values respectively.
* df.std(), df.var(): Methods to get standard deviation and variance respectively.
* df.mad() : Method to get mean absolute deviation 
* df.prod() : Method to get product of the all items
* df.sum() : Method to get sum of all the items.

Groupby: Split, Apply ,Combine
* Split : Breaks and groups data frame depending on the value of the specified key.
* Apply : Computes some functions, usually an aggregate, transformation or filtering, within individual groups.
* Combine : Merges the results of these operations into an output array.

In [17]:
import numpy as np
rng = np.random.RandomState(0) # Container for the Mersenne Twister 
                               # pseudo-random number generator.
                               # In this case Seed = 0
df = pd.DataFrame(
        {'key' : [ 'A', 'B', 'C', 'A', 'B', 'C'], 
         'data1': range(6), 
         'data2': rng.randint(0, 10, 6)
        })
df


,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


## Grouping using <code>.groupby()</code>

In [18]:
#Grouping by key using the mean to aggregate
df.groupby('key').mean()


,data1,data2
key,,
A,1.5,4.0
B,2.5,3.5
C,3.5,6.0


In [19]:
import seaborn as sns # statistical data visualization
titanic = sns.load_dataset('titanic') # load_dataset looks for online csv files 
                                    #on https://github.com/mwaskom/seaborn-data
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [20]:
#Grouping by sex using the sum to aggregate
titanic.groupby('sex').sum()

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
sex,,,,,,,,
female,233,678,7286.00,218,204,13966.6628,0,126
male,109,1379,13919.17,248,136,14727.2865,537,411


In [21]:
#Grouping by class using the mean to aggregate
titanic.groupby('class').mean()

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
class,,,,,,,,
First,0.629630,1.0,38.233441,0.416667,0.356481,84.154687,0.550926,0.504630
Second,0.472826,2.0,29.877630,0.402174,0.380435,20.662183,0.538043,0.565217
Third,0.242363,3.0,25.140620,0.615071,0.393075,13.675550,0.649695,0.659878


## Looking at the groups with <code>.get_group()</code>

Gruping data by 'sex' and accessing the group 'female'

In [22]:
titanic.groupby('sex').get_group('female').head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


Grouping by 'sex' and 'class' and accessing the group 'female' 'First'

In [23]:
titanic.groupby(['sex','class']).get_group(('female','First')).head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
11,1,1,female,58.0,0,0,26.5500,S,First,woman,False,C,Southampton,yes,True
31,1,1,female,NaN,1,0,146.5208,C,First,woman,False,B,Cherbourg,yes,False
52,1,1,female,49.0,1,0,76.7292,C,First,woman,False,D,Cherbourg,yes,False


Now aggregating this group using the mean and corroborating the plot of the movie 'Titanic'

In [24]:
titanic.groupby(['sex','class']).get_group(('female','First')).mean(numeric_only=True)

survived        0.968085
pclass          1.000000
age            34.611765
sibsp           0.553191
parch           0.457447
fare          106.125798
adult_male      0.000000
alone           0.361702
dtype: float64

In [25]:
titanic.groupby(['sex','class']).get_group(('male','Third')).mean(numeric_only=True)

survived       0.135447
pclass         3.000000
age           26.507589
sibsp          0.498559
parch          0.224784
fare          12.661633
adult_male     0.919308
alone          0.760807
dtype: float64

## <code>.agg</code> (short for aggregate) affords more control 
Aggregation does not works for all columns, we need to select colums for which the aggregation makes sence. There is a minimum from {20, 4} but not from {Female, Male}

In [26]:
#Aggregation
# for instance look at min age of who survived, min pclass, min fare
titanic.groupby('survived').agg({"age":['min'],"pclass":['min'],"fare":['min']}) 

,age,pclass,fare
,min,min,min
survived,,,
0,1.00,1,0.0
1,0.42,1,0.0


Multiple aggregations

In [27]:
#Aggregation multiple
# for instance look at min max age, class and fare of who survived
titanic.groupby('survived').agg({"age":['min','max'],"pclass":['min','max'],"fare":['min','max']}) 

age       pclass     fare          
           min   max    min max  min       max
survived                                      
0         1.00  74.0      1   3  0.0  263.0000
1         0.42  80.0      1   3  0.0  512.3292

Aggregating a single column, age

In [28]:
#Aggregation of column
titanic.groupby('survived').age.agg(['min','mean','max']) # age: min, mean, max


,min,mean,max
survived,,,
0,1.00,30.626179,74.0
1,0.42,28.343690,80.0


Different aggregations per column

In [29]:
#Aggregation of column
titanic.groupby('survived').agg({'age':['min','mean','max'],'fare':'mean'}) # age: min, mean, max

age                        fare
           min       mean   max       mean
survived                                  
0         1.00  30.626179  74.0  22.117887
1         0.42  28.343690  80.0  48.395408

## Tranformation using <code>.transform</code>
While aggregation must return a reduced version of the data, transformation can operate on entire grouped data but one group at a time. In the example below, the mean of a particular group is substracted from each element of the group.

In [30]:
# Transformation
# titanic2 is created to operate "mean()" on only number columns
titanic2 = titanic[["sex", "age", "fare"]]
titanic2.head()

,sex,age,fare
0,male,22.0,7.2500
1,female,38.0,71.2833
2,female,26.0,7.9250
3,female,35.0,53.1000
4,male,35.0,8.0500


In [31]:
titanic2.groupby('sex').mean()

,age,fare
sex,,
female,27.915709,44.479818
male,30.726645,25.523893


In [32]:
titanic2.groupby("sex").transform(lambda x: x - x.mean()).head()

,age,fare
0,-8.726645,-18.273893
1,10.084291,26.803482
2,-1.915709,-36.554818
3,7.084291,8.620182
4,4.273355,-17.473893


## <code>Apply()</code>

This method lets us apply an arbitrary function to the group results. This method should take a data frame and return either a Pandas object (e.g., DataFrame or Series) or a scalar.



In [33]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [34]:
def norm_by_data2(x):
    x['data1'] = x['data1']/x['data2'].sum()
    return x

df.groupby('key').get_group('A')
# in example below, A[data1]=A[data1]/A[data2].sum, results in [0] 0/8=0 and [3] 3/8=0.375 


,key,data1,data2
0,A,0,5
3,A,3,3


In [35]:
df.groupby('key').get_group('B')
# in example below, B[data1]=B[data1]/B[data2].sum, results in [1] 1/7=0.14 and [4] 4/7=0.57

,key,data1,data2
1,B,1,0
4,B,4,7


In [36]:
df.groupby('key').apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3
3,A,0.375000,3
4,B,0.571429,7
5,C,0.416667,9


# Merging data with <code>.merge()</code>

To combine datasets by linking rows using one or more keys, we use <code>.merge()</code> operation. <br>

In [37]:
df1 = pd.DataFrame({'key1': ['b', 'b', 'a', 'c', 'a', 'a',  'b'], 'data1': range(7)})
df1

,key1,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [38]:
df2 = pd.DataFrame ({'key1': [ 'a', 'b', 'd'], 'data2': range(3)})
df2

,key1,data2
0,a,0
1,b,1
2,d,2


In [39]:
pd.merge(df1, df2) 

,key1,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


This <code>.merge()</code> merges by using the overlapping column names as the keys, <code>key1</code> in this case. This is because we did not specify which column to **join** explicitly. We can specify which column is the key for the join as follows.



In [60]:
pd.merge(df1, df2, on = 'key1')


,key1,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


By default the behaviour of <code>.merge()</code> is to conduct an __inner join__, however that can also be modified as follows:

Other options : <br>
* <code>how = 'inner' </code>: Use only the key combinations observed in both tables.
* <code>how = 'outer' </code>: Use all the key combinations observed in both tables together.
* <code>how = 'left' </code>: Use all key combinations found in the left table.
* <code>how = 'right' </code> : Use all key combinations found in the right table.


In [41]:
pd.merge(df1, df2, how = 'outer')

,key1,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [42]:
pd.merge(df1, df2, how = 'left')

,key1,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


# Pivot Tables

A pivot table is a similar operation that is commonly seen in spreadsheets and other programs that operate on tabular data. The pivot table takes simple column-wise data as input and groups the entries into a two dimensional table that provides a multidimensional summarization of the data. <br>

In [43]:
import seaborn as sns # statistical data visualization
titanic = sns.load_dataset('titanic') # load_dataset looks for online csv files 
                                    #on https://github.com/mwaskom/seaborn-data
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [44]:
titanic.pivot_table(values=['survived','fare'], index = 'sex', columns = 'class')

fare                        survived                    
class        First     Second      Third     First    Second     Third
sex                                                                   
female  106.125798  21.970121  16.118810  0.968085  0.921053  0.500000
male     67.226127  19.741782  12.661633  0.368852  0.157407  0.135447

The aggfunc keyword controls what type of aggregation is applied, which is mean by default. As in the GroupBy, the aggregation specification can be a string representing one of several common choices(‘sum’, ‘mean’, ‘count’, ‘min’, ‘max’, etc.) or a function that implements an aggregation (np.sum(), min(), sum(), etc.).

Additionally, it can be specified as dictionary mapping a column to any of the above desired options: 

In [45]:
titanic.pivot_table(index = 'sex', columns = 'class', aggfunc = {'survived' : sum, 'fare' : 'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

# Data cleaning and preparation

Data cleaning is the process of removing bad data in a dataset. This bad data includes incorrect and improperly formatted data as well as duplicated and missing data. 

In [46]:
# Example of a student survey dataset which includes incorrect and improperly formatted data.

csv_path = '../Data/Student_Survey.csv'

df = pd.read_csv (csv_path)
df

,Unnamed: 0,Year,Location,Education,Sample_Size,Satisfactory
0,0,2017.0,Putnam,Middle School,659.0,Y
1,1,2018.0,Lexington,Middle School,649.0,N
2,2,2018.0,Lexington,Middle School,435.0,N
3,3,2017.0,Berkeley,NaN,NaN,NaN
4,4,NaN,Berkeley,High School,228.0,Y
5,5,2018.0,Berkeley,Middle School,20.0,NaN
6,6,2018.0,Washington,High School,437.0,N
7,7,NaN,Tremont,High School,NaN,Y
8,8,2016.0,Tremont,High School,220.0,Y


Let's take a look at the dataset. There are seven NA values in all columns. By using ```isnull``` function, pandas recognizes all missing value and return ```True```  

In [47]:
# Recognizing missing values

print (df.isnull())


   Unnamed: 0   Year  Location  Education  Sample_Size  Satisfactory
0       False  False     False      False        False         False
1       False  False     False      False        False         False
2       False  False     False      False        False         False
3       False  False     False       True         True          True
4       False   True     False      False        False         False
5       False  False     False      False        False          True
6       False  False     False      False        False         False
7       False   True     False      False         True         False
8       False  False     False      False        False         False


### ```dropna``` method

```dropna``` is a method to filter missing data. Sometimes you need to work on only correct data and want to omit others.  

In [48]:
# Dropping all missing data by omitting rows and/or columns 
# which include missing data, Default delete rows.

df.dropna()

,Unnamed: 0,Year,Location,Education,Sample_Size,Satisfactory
0,0,2017.0,Putnam,Middle School,659.0,Y
1,1,2018.0,Lexington,Middle School,649.0,N
2,2,2018.0,Lexington,Middle School,435.0,N
6,6,2018.0,Washington,High School,437.0,N
8,8,2016.0,Tremont,High School,220.0,Y


In [49]:
# Dropping rows and columns which are all NA (In this example, there is no row includes all NA)

df.dropna(how = 'all')    #For rows

,Unnamed: 0,Year,Location,Education,Sample_Size,Satisfactory
0,0,2017.0,Putnam,Middle School,659.0,Y
1,1,2018.0,Lexington,Middle School,649.0,N
2,2,2018.0,Lexington,Middle School,435.0,N
3,3,2017.0,Berkeley,NaN,NaN,NaN
4,4,NaN,Berkeley,High School,228.0,Y
5,5,2018.0,Berkeley,Middle School,20.0,NaN
6,6,2018.0,Washington,High School,437.0,N
7,7,NaN,Tremont,High School,NaN,Y
8,8,2016.0,Tremont,High School,220.0,Y


In [50]:
df.dropna(axis = 1, how ='all')     #For columns

,Unnamed: 0,Year,Location,Education,Sample_Size,Satisfactory
0,0,2017.0,Putnam,Middle School,659.0,Y
1,1,2018.0,Lexington,Middle School,649.0,N
2,2,2018.0,Lexington,Middle School,435.0,N
3,3,2017.0,Berkeley,NaN,NaN,NaN
4,4,NaN,Berkeley,High School,228.0,Y
5,5,2018.0,Berkeley,Middle School,20.0,NaN
6,6,2018.0,Washington,High School,437.0,N
7,7,NaN,Tremont,High School,NaN,Y
8,8,2016.0,Tremont,High School,220.0,Y


### ```fillna``` method 

```fillna``` is a method to fill missing data by any number or value.  

In [51]:
# filling missing data by '100' in 'Sample_Size' column. 

print (df['Sample_Size'])
df.fillna({'Sample_Size': 100})

0    659.0
1    649.0
2    435.0
3      NaN
4    228.0
5     20.0
6    437.0
7      NaN
8    220.0
Name: Sample_Size, dtype: float64


,Unnamed: 0,Year,Location,Education,Sample_Size,Satisfactory
0,0,2017.0,Putnam,Middle School,659.0,Y
1,1,2018.0,Lexington,Middle School,649.0,N
2,2,2018.0,Lexington,Middle School,435.0,N
3,3,2017.0,Berkeley,NaN,100.0,NaN
4,4,NaN,Berkeley,High School,228.0,Y
5,5,2018.0,Berkeley,Middle School,20.0,NaN
6,6,2018.0,Washington,High School,437.0,N
7,7,NaN,Tremont,High School,100.0,Y
8,8,2016.0,Tremont,High School,220.0,Y


In [52]:
# filling missing data in both columns 'Year' by 2015 and 'Sample_Size' by '100'.

df.fillna({'Year': 2015, 'Sample_Size': 100})

,Unnamed: 0,Year,Location,Education,Sample_Size,Satisfactory
0,0,2017.0,Putnam,Middle School,659.0,Y
1,1,2018.0,Lexington,Middle School,649.0,N
2,2,2018.0,Lexington,Middle School,435.0,N
3,3,2017.0,Berkeley,NaN,100.0,NaN
4,4,2015.0,Berkeley,High School,228.0,Y
5,5,2018.0,Berkeley,Middle School,20.0,NaN
6,6,2018.0,Washington,High School,437.0,N
7,7,2015.0,Tremont,High School,100.0,Y
8,8,2016.0,Tremont,High School,220.0,Y


## Removing Duplicates

Sometimes in a DataFrame, you have duplicate rows and you need to remove them. See dataframe below with few duplicate rows.

In [53]:
# Creating a DataFrame with few duplicated rows
raw_data = [['Clark','Kent',28,3,70],['Bruce','Banner',38,4,25],['Clark','Kent',28,3,70],['Tony','Stark',42,24,94],['Hal','Jordan',25,31,57],['Bruce','Wayne',32,2,62],
            ['Tony','Stark',42,24,94],['Tony','Stark',42,24,94]]
df = pd.DataFrame (raw_data, columns = ['first_name', 'last_name','age','preTestScore','postTestScore'])
df

,first_name,last_name,age,preTestScore,postTestScore
0,Clark,Kent,28,3,70
1,Bruce,Banner,38,4,25
2,Clark,Kent,28,3,70
3,Tony,Stark,42,24,94
4,Hal,Jordan,25,31,57
5,Bruce,Wayne,32,2,62
6,Tony,Stark,42,24,94
7,Tony,Stark,42,24,94


```duplicated()``` method returns a boolean value whether each row is a duplicate. In this DataFrame, the value for number 5 is 'True' and says row number 5 is duplicated.

In [54]:
df.duplicated()

0    False
1    False
2     True
3    False
4    False
5    False
6     True
7     True
dtype: bool

```drop_duplicates()``` method returns DataFrame with duplicate rows removed

In [55]:
df.drop_duplicates()

,first_name,last_name,age,preTestScore,postTestScore
0,Clark,Kent,28,3,70
1,Bruce,Banner,38,4,25
3,Tony,Stark,42,24,94
4,Hal,Jordan,25,31,57
5,Bruce,Wayne,32,2,62


# References
1. Seaborn, statistical data visualization, https://seaborn.pydata.org
2. Data repo for Seaborn examples, https://github.com/mwaskom/seaborn-data
3. Pivot Tables, https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html
4. Reshaping, Pivot, Stack, unstack, https://pandas.pydata.org/pandas-docs/stable/reshaping.html

_Last updated on 9.5.2025 1:18pm<br>
(C) 2025 Complex Adaptive Systems Laboratory, all rights reserved._